**Тестовая задача 7. Работа с графовыми БД**

1. Импортируем библиотеки:

In [1]:
#для получения файла с Яндекс Диска
import requests
import pprint
import urllib.parse

#для удобства просмотра результатов
import pandas as pd

#драйвер для соединения с базой
from neo4j import GraphDatabase

#для визуализации
from yfiles_jupyter_graphs import GraphWidget
#для генерации hex значений цветов 
from random import choice

#для API
from flask import Flask, request, make_response
import json

2. Получаем файл с Яндекс диска в виде объекта:

Примечание: данная ссылка не соответствует той, что была дана для скачивания файла, т.к. Яндекс ограничил кол-во скачиваний.
Мне пришлось пересохранить файл и загрузить на свой Я.Диск.

In [2]:
target_url = "https://disk.yandex.ru/d/XQ0WpAsDeS7yKw"


url = "https://cloud-api.yandex.net/v1/disk/public/resources?public_key=" + urllib.parse.quote(target_url, safe="")

response_data = requests.get(url)
response_json = response_data.json()
link_to_file = response_json['file']

if response_data.headers["content-type"] in ["application/json; charset=utf-8", "application/json"]:
    print('Итак, мы вытащили ссылку на файл:')
    print(link_to_file)
#   print(pprint.pformat(responseJson)) эту строку нужно раскомментировать чтобы просмотреть Json объект целиком
else:
    print(response_data)


Итак, мы вытащили ссылку на файл:
https://downloader.disk.yandex.ru/disk/4d3f3e40b4aee1292f071a8d807bf146f6b41bdf765138741ea3570fe8ba9738/6407dbde/cnCdw-2I-y-0tP07jc6mB_YD9m1xJVbkCZQV_bZ1haEqZORVS6i-Ga2QtlP1QgSP-9meee3b6JTSrShpDVjT6A%3D%3D?uid=0&filename=data_test_copy.csv&disposition=attachment&hash=&limit=0&content_type=text%2Fplain&owner_uid=0&fsize=548614&hid=47c23cb742bbee6175b00517d3312bf2&media_type=spreadsheet&tknv=v2&etag=4a1feab956f2e434224abbfce06d4cca


3. Перед тем как грузить файл в базу, бегло просмотрим его. Прочитаем данные из файла в DataFrame и проверим, что же мы вытащили:

In [3]:
df = pd.read_csv(link_to_file, sep=';')

df.head()

,id события,ФИО участника события 1,ФИО участника события 2
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович
3,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович
4,571,Мухтарова Алена Яковлевна,Щербатенко Ольга Робертовна


4. Подключимся к БД. Для этого создадим экземпляр драйвера.

In [6]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j1", "neo4j1neo4j1")) 
#в качестве аргументов драйвер принимает адрес сервера БД, имя пользователя и пароль

5. Протестируем, распознаёт ли БД наш файл

In [8]:
def test_csv(tx, csv_link):
        result = tx.run("""LOAD CSV WITH HEADERS FROM $csv_link AS row FIELDTERMINATOR ';'
                        RETURN
                        toInteger(row['id события']),
                        row['ФИО участника события 1'],
                        row['ФИО участника события 2']
                        LIMIT 5""" , csv_link = csv_link
                        )
        records = list(result)  
        return records


with driver.session() as session:
            records = session.execute_read(test_csv, link_to_file)

(pd.DataFrame(records)).head(5)

,0,1,2
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович
3,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович
4,571,Мухтарова Алена Яковлевна,Щербатенко Ольга Робертовна


6. Перед тем как загружать данные в БД, создадим ограничения, где пропишем какие свойства узлов должны быть уникальными. Для события это будет id, для участников - ФИО.

In [9]:
def events_constraint(tx):
    result = tx.run("""CREATE CONSTRAINT Event_id IF NOT EXISTS
                        FOR (x:Event)
                        REQUIRE x.eventId IS UNIQUE
                       """ 
                        )
def person_constraint(tx):
    result = tx.run("""CREATE CONSTRAINT Person_name IF NOT EXISTS
                        FOR (x:Person)
                        REQUIRE x.name IS UNIQUE
                       """ 
                        )
    
with driver.session(database="mytest") as session:
    session.execute_write(events_constraint)

7. Загрузим события

In [ ]:
def load_events(tx, csv_link):
     result = tx.run("""CALL {
                        LOAD CSV WITH HEADERS FROM $csv_link AS row FIELDTERMINATOR ';'
                        MERGE (e:Event {eventId: toInteger(row['id события'])})
                        }
                       """ , csv_link = csv_link
                        )
    
with driver.session(database="mytest") as session:
            session.execute_write(load_events, link_to_file)

8. Загрузим участников

In [ ]:
def load_people(tx, csv_link):
     result = tx.run("""CALL {
                        LOAD CSV WITH HEADERS FROM $csv_link AS row FIELDTERMINATOR ';'
                        MERGE (p1:Person {name: toString(row['ФИО участника события 1'])})
                        MERGE (p2:Person {name: toString(row['ФИО участника события 2'])})
                        }
                       """ , csv_link = csv_link
                        )
with driver.session(database="mytest") as session:
            session.execute_write(load_people, link_to_file)

9. Создадим связи между событиями и участниками (связь направлена от участика к событию, в котором он участвовал)

In [ ]:
def create_edges(tx, csv_link):
    result =  tx.run("""CALL {
                        LOAD CSV WITH HEADERS FROM $csv_link AS row FIELDTERMINATOR ';'
                        MATCH (p1:Person {name: toString(row['ФИО участника события 1'])})
                        MATCH (p2:Person {name: toString(row['ФИО участника события 2'])})
                        MATCH  (e:Event {eventId: toInteger(row['id события'])})
                        MERGE (p1)-[x:PARTICIPATED]->(e)
                        MERGE (p2)-[y:PARTICIPATED]->(e)
                        }
                       """ , csv_link = csv_link
                        )
with driver.session(database="mytest") as session:
            session.execute_write(create_edges, link_to_file)

Посмотрим, кто принимал участие в событиях больше всех:

In [11]:
def most_active_person(tx):
    result =  tx.run("""
                        MATCH (p:Person)-[r:PARTICIPATED]->(e:Event)
                        WITH p, count(r) as rels, collect(e) as events
                        WHERE rels > 1
                        RETURN p.name AS name, rels AS rels
                        ORDER BY SIZE(events) DESC LIMIT 1
                       """
                        )
    return result.values("name", "rels")

with driver.session(database="mytest") as session:
            result_list = session.execute_write(most_active_person)    

print("Самый активный участник - " + str(result_list[0][0]) + ". Количество событий участника - " + str(result_list[0][1]))




Самый активный участник - Ахромеева Алина Ивановна. Количество событий участника - 50


10. Создадим связи между участниками. 
Будем считать, что если 2 человека участвовали в одном событии, то они знакомы. 

In [ ]:
def create_friends(tx, csv_link):
    result =  tx.run("""CALL {
                        LOAD CSV WITH HEADERS FROM $csv_link AS row FIELDTERMINATOR ';'
                        MATCH (p1:Person {name: toString(row['ФИО участника события 1'])})
                        MATCH (p2:Person {name: toString(row['ФИО участника события 2'])})
                        MERGE (p1)-[x:KNOWS]-(p2)
                        MERGE (p2)-[y:KNOWS]-(p1)
                        }
                       """ , csv_link = csv_link
                        )
with driver.session(database="mytest") as session:
            session.execute_write(create_friends, link_to_file)

Проверим, у кого больше всех знакомств:

In [10]:
def most_friends_person(tx):
    result =  tx.run("""
                        MATCH (p:Person)-[r:KNOWS]-(f:Person)
                        WITH p, count(r) as friends
                        RETURN p.name AS name, friends
                        ORDER BY friends DESC LIMIT 1
                       """
                        )
    return result.values("name", "friends")

with driver.session(database="mytest") as session:
            result_list = session.execute_read(most_friends_person)    

print("Участник у кого больше всего знакомств - " + str(result_list[0][0]) + ". Количество знакомств участника - " + str(result_list[0][1]))

Участник у кого больше всего знакомств - Ахромеева Алина Ивановна. Количество знакомств участника - 50


Больше всех знакомых у той же персоны, которая принимала участие в большем количестве событий.

10. Визуализация
При помощи библиотеки yfiles_jupyter_graphs создадим виджет с визуализацией прямо в Jupyter блокноте.
Для начала получим из БД граф содержащий все узлы, связанные знакомствами (связью KNOWS).

In [12]:
with driver.session(database="mytest") as session:
            graph = session.run("""MATCH p=()-[r:KNOWS]-() RETURN p""").graph()

Настроим представление виджета и запустим его (будем показывать его по умолчанию в органическом представлении графа и с выводом ФИО в качестве подписи к узлу):

In [13]:
w = GraphWidget(graph = graph)
w.set_graph_layout("organic")
w.set_node_label_mapping(lambda index, node: node["properties"]["name"] if node["properties"]["label"] == "Person" else node["properties"]["label"])
w.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

Визуально мы можем выделить 6 сообществ. Остальные "сообщества" представлены парами участников.

10. Проведем анализ нашего графа при помощи библиотеки БД Neo4j Graph Data Science

10.1 Для применения алгоритмов в первую очередь необходимо создать в БД in-memory графы. Создадим граф содержащий участников, связанных связьу KNOWS, укажем что связи будут ненаправленные.

In [14]:
def create_graph_knows(tx):
    result =  tx.run("""CALL gds.graph.project(
                            'interactions',
                            'Person',
                            {KNOWS:
                                {
                                orientation:'UNDIRECTED'
                                }
                            }
                            )
                     """
                        )
    return list(result)

def drop_draph_knows(tx): #используем эту функцию когда нужно будет удалить ненужные графы после работы
    result = tx.run("""CALL gds.graph.drop('interactions')""") 

with driver.session(database="mytest") as session:
            result = session.execute_write(create_graph_knows) 


Создадим также граф содержащий как участников, так и события + связь участников с событиями PARTICIPATED. 

Просмотрим наш список графов, чтобы удостовериться, что всё правильно создалось:

In [ ]:
def graphs_list(tx):
    result =  tx.run("""CALL gds.graph.list()
                            
                     """
                        )
    return list(result)


with driver.session(database="mytest") as session:
            result = session.execute_read(graphs_list) 
    
pd.DataFrame(result).head(10)

10.2 Рассчитаем некоторые меры центральности (centrality) для узлов нашей сети.

1. PageRank
Алгоритм PageRank измеряет важность каждого узла в графе на основе количества входящих отношений и важности соответствующих исходных узлов. Основное предположение состоит в том, что узел важен настолько, насколько важны узлы, которые ссылаются на него. (Применяется обычно для веб страниц). В данной библиотеке может применяться и к ненаправленным графам.
https://neo4j.com/docs/graph-data-science/current/algorithms/page-rank/

In [15]:
def page_rank(tx):
    result = tx.run(
                    """CALL gds.pageRank.stream('interactions')
                    YIELD nodeId, score
                    RETURN gds.util.asNode(nodeId).name AS name, score
                    ORDER BY score DESC, name ASC
                    LIMIT 10"""
                    )
    return list(result)

with driver.session(database="mytest") as session:
            result = session.execute_read(page_rank) 

result
pd.DataFrame(result).head()

,0,1
0,Ахромеева Алина Ивановна,22.602141
1,Башнина Антонина Глебовна,6.702704
2,Медведева Дарья Алексеевна,3.169496
3,Диомидов Игорь Ильдарович,2.496732
4,Зимнухова Карина Даниловна,2.496732


Вновь участник Ахромеева Алина Ивановна занимает лидирующую позицию.

2. Степень посредничества
Степень посредничества (betweenness centrality) является мерой того, как часто кратчайшие пути проходят через данную вершину.
https://neo4j.com/docs/graph-data-science/current/algorithms/betweenness-centrality/

In [16]:
def betweenness(tx):
    result = tx.run(
                    """CALL gds.betweenness.stream('interactions')
                    YIELD nodeId, score
                    RETURN gds.util.asNode(nodeId).name AS name, score
                    ORDER BY score DESC, name ASC
                    LIMIT 10"""
                    )
    return list(result)

with driver.session(database="mytest") as session:
            result = session.execute_read(betweenness) 


(pd.DataFrame(result, columns=['name', 'score'])).head(10)
        

,name,score
0,Ахромеева Алина Ивановна,1225.0
1,Башнина Антонина Глебовна,91.0
2,Шолохов Игорь Робертович,41.0
3,Диомидов Игорь Ильдарович,38.0
4,Зимнухова Карина Даниловна,38.0
5,Двигубская Валентина Геннадьевна,37.0
6,Бугайчук Роман Эдуардович,36.0
7,Анихнова Тамара Руслановна,35.0
8,Радионова Тамара Ярославовна,35.0
9,Медведева Дарья Алексеевна,15.0


3. Степень центральности
Близость к центру или Степень центральности (Degree centrality) – показывает, кто является наиболее активным узлом в сети. Измеряется количеством связей с другими узлами в сети.

In [17]:
def degree(tx):
    result = tx.run(
                    """CALL gds.degree.stream('interactions')
                    YIELD nodeId, score
                    RETURN gds.util.asNode(nodeId).name AS name, score
                    ORDER BY score DESC, name ASC
                    LIMIT 10"""
                    )
    return list(result)

with driver.session(database="mytest") as session:
            result = session.execute_read(degree) 


(pd.DataFrame(result, columns=['name', 'score'])).head(10)

,name,score
0,Ахромеева Алина Ивановна,50.0
1,Башнина Антонина Глебовна,14.0
2,Медведева Дарья Алексеевна,6.0
3,Диомидов Игорь Ильдарович,5.0
4,Зимнухова Карина Даниловна,5.0
5,Шолохов Игорь Робертович,4.0
6,Двигубская Валентина Геннадьевна,3.0
7,Пафомова Кира Вадимовна,3.0
8,Анихнова Тамара Руслановна,2.0
9,Батиевская Ангелина Романовна,2.0


В данном случае алгоритм выдал нам то же значение, что мы и получили ранее при помощи запроса - участника, у которого больше всего знакомств.

10.3 Выделение сообществ

1. Метод Лувена https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/

Выделим сообщества:

In [18]:
def louvain(tx):
    result = tx.run(
                    """CALL gds.louvain.stream(
                    'interactions')
                    YIELD nodeId, communityId
                    RETURN communityId, COUNT(DISTINCT nodeId) AS members
                    ORDER BY members DESC
                    LIMIT 10"""
                    )
    return list(result)

with driver.session(database="mytest") as session:
            result = session.execute_read(louvain) 

louvain_df = pd.DataFrame(result, columns=['community_id', 'members'])
louvain_df.head(10)


,community_id,members
0,8207,51
1,8327,15
2,8489,13
3,8647,13
4,8859,7
5,2140,6
6,4950,2
7,4949,2
8,4946,2
9,4953,2


Мы видим, что на основании знакомств у нас есть 6 сообществ - то же самое, что мы могли увидеть и визуально. Остальные сообщества являются просто парами которые принимали участие в одном событии.

Заберем себе id первых 6 сообществ, они нам понадобятся для цветовой идентификации при визуализации.

In [22]:
louv_comm_id_list = louvain_df['community_id']. tolist()[:6]
louv_comm_id_list

[8207, 8327, 8489, 8647, 8859, 2140]

Присвоим участникам сообществ соответствующие метки:

In [19]:
def louvain_property(tx):
    result = tx.run(
                    """CALL gds.louvain.write(
                        'interactions', 
                        {
                            writeProperty: 'louv_community'
                       }
                                            )
                        YIELD communityCount, modularity, modularities"""
                    )
    return list(result)

with driver.session(database="mytest") as session:
            result = session.execute_write(louvain_property) 


(pd.DataFrame(result, columns=['community_count', 'modularity', 'modularities'])).head()


,community_count,modularity,modularities
0,4903,0.999679,"[0.997490159999997, 0.9994830599999963, 0.9996..."


При этом в режиме записи возвращается строка содержащая информацию о количестве сообществ, модулярности, и истории изменения модулярности в ходе исполнения алгоритма. Мы видим, что у нас есть 4903 сообщества и очень высокая модулярность графа - почти 1. Т.е. сообщества почти не связаны между собой.

2. Алгоритм распространения метки - Label propagation algorithm

Произведем также выделение сообществ при помощи другого алгоритма (Label propagation algorithm). 

In [20]:
def label_propagation(tx):
    result = tx.run(
                    """CALL gds.labelPropagation.stream(
                     'interactions'
                        )
                    YIELD nodeId, communityId
                    RETURN communityId, COUNT(DISTINCT nodeId) AS members
                    ORDER BY members DESC
                    LIMIT 10"""
                    )
    return list(result)

with driver.session(database="mytest") as session:
            result = session.execute_read(label_propagation) 


label_propagation_df = pd.DataFrame(result, columns=['community_id', 'members'])
label_propagation_df.head(10)




,community_id,members
0,20162,51
1,20282,15
2,1698,13
3,20814,7
4,1766,6
5,21735,5
6,20602,5
7,2891,3
8,4950,2
9,4949,2


Алгоритм распространения метки выделил 8 сообществ помимо пар участников. Также сохраним эти метки в списке.

In [21]:
label_propagation_id_list = label_propagation_df['community_id']. tolist()[:8]
label_propagation_id_list

[20162, 20282, 1698, 20814, 1766, 21735, 20602, 2891]

In [ ]:
Присвоим участникам сообществ также метки с id сообществ, выделенных при помощи этого алгоритма:

In [23]:
def label_propagation_property(tx):
    result = tx.run(
                    """CALL gds.labelPropagation.write(
                       'interactions', 
                           {
                               writeProperty: 'lbl_prop_community'
                           }
                        ) 
                        YIELD communityCount, ranIterations, didConverge"""
                    )
    return list(result)

with driver.session(database="mytest") as session:
            result = session.execute_write(label_propagation_property) 


(pd.DataFrame(result, columns=['community_count', 'ranIterations', 'didConverge'])).head()

,community_count,ranIterations,didConverge
0,4905,4,True


10.4 Визуальное сравнение сообществ, выделенных двумя алгоритмами.

Вернёмся к виджету для визуализации. Настроим цветовую индикацию сообществ и выведем 2 визуализации.
Для начала создадим функцию которая присваивает некоторые значения цветов элементам списка.

In [24]:
def give_colors_to_list(our_list:list):
    """Функция принимает список числовых значений int и возвращает словарь "ключ-цвет",
        где ключ - число из переданного функции списка"""
    color_list = []
    for i in our_list:
        color = "#"+''.join([choice('0123456789abcdef') for j in range(6)])
        color_list.append(color)
    color_dictionary = dict(zip(our_list, color_list))
    return color_dictionary

И создадим 2 словаря для цветовой идентификации наших сообществ полученных при помощи различных алгоритмов:

In [26]:
louv_colors_dict = give_colors_to_list(louv_comm_id_list)
label_prop_colors_dict = give_colors_to_list(label_propagation_id_list)

Далее получим из базы обновлённый граф (т.к. мы добавили узлам-персонам свойства с номерами id сообществ).

In [27]:
with driver.session(database="mytest") as session:
            graph = session.run("""MATCH p=()-[r:KNOWS]-() RETURN p""").graph()

Выведем виджет с визуализацией графа, назначив цветовую идентификацию сообществам выделенным при помощи Лувенского алгоритма.

In [28]:
w_louvain = GraphWidget(graph = graph)

def custom_color_mapping(index, node):
    louv_community_number = node.get('properties', {}).get('louv_community')
    if louv_community_number in louv_comm_id_list:
        color = louv_colors_dict[louv_community_number]
        return color
    else:
        return "grey"

w_louvain.set_graph_layout("organic")
w_louvain.set_node_label_mapping(lambda index, node: node["properties"]["name"] if node["properties"]["label"] == "Person" else node["properties"]["label"])
w_louvain.set_node_color_mapping(custom_color_mapping)
w_louvain.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

Мы видим те же 6 сообществ, которые мы и определили визуально.

Сделаем то же самое, выделив цветами сообщества, полученные при помощи алгоритма распространения метки.

In [29]:
w_lable_prop = GraphWidget(graph = graph)

def custom_color_mapping(index, node):
    lbl_prop_community_number = node.get('properties', {}).get('lbl_prop_community')
    if lbl_prop_community_number in label_propagation_id_list:
        color = label_prop_colors_dict[lbl_prop_community_number]
        return color
    else:
        return "grey"

w_lable_prop.set_graph_layout("organic")
w_lable_prop.set_node_label_mapping(lambda index, node: node["properties"]["name"] if node["properties"]["label"] == "Person" else node["properties"]["label"])
w_lable_prop.set_node_color_mapping(custom_color_mapping)
w_lable_prop.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

Согласно цветовой идентификации, мы видим что алгоритм распространения метки определил первое сообщество как 3.

Визуально мы также видим, что некоторые сообщества представлены в форме "звезды" - т.е. они представляют собой просто множество участников, связанных одним общим знакомым. Зададим вопрос - есть ли у нас "компанейские" сообщества, т.е. в каких из них присутствуют участники, у которых знакомые могут также знать друг друга? Рассчитаем для этого коэффициэнт кластеризации для каждого из выделенных Лувенским методом сообществ.
Для этого создадим in-memory граф, включающий помимо участников и связей между ними также свойство "louv_community", которого раньше у нас не было.

In [ ]:
def create_louv_interactions_graph(tx):
    result = tx.run(
                    """CALL gds.graph.project(
                            'louv_interactions',
                            'Person',
                            {KNOWS:
                                {
                                orientation:'UNDIRECTED'
                                }
                            },
                            {nodeProperties: 'louv_community'} 
                            )"""
                    )
    return list(result)

with driver.session(database="mytest") as session:
            result = session.execute_write(create_louv_interactions_graph) 

Далее, разобьём наш граф на подграфы согласно id сообществ (берем только 6 интересующих нас сообществ)

In [34]:
def create_louv_subgraphs(tx, name, given_id):
    """Т.к. функция gds.beta.graph.project.subgraph не поддерживает передачу динамчиеских параметров, 
    передаём их в запрос при помощи конкатенации, для этого предварительно экранируем"""
    name = str(name)
    escaped_name = name.replace("\\u0060", "`").replace("`", "``")
    given_id = str(given_id)
    escaped_given_id = given_id.replace("\\u0060", "`").replace("`", "``")
    result = tx.run(
                     """CALL gds.beta.graph.project.subgraph( 
                        '"""+escaped_name+"""',
                        'louv_interactions',
                        'n.louv_community = """+given_id+"""',
                        '*'
                        )
                        YIELD graphName, fromGraphName, nodeCount, relationshipCount
                        """,name = name, given_id = given_id
                    )

for item in louv_comm_id_list:
    with driver.session(database="mytest") as session:
        result = session.execute_write(create_louv_subgraphs, name = item, given_id = item) 

Проверка: посмотрим на список графов

In [ ]:
with driver.session(database="mytest") as session:
            result = session.execute_read(graphs_list) 
    
pd.DataFrame(result).head(10)

Вычислим для подграфов коэффициент кластеризации.

In [36]:
def calculate_avg_сlustering_сoefficient(tx, graph_name):
    graph_name = str(graph_name)
    escaped_graph_name = graph_name.replace("\\u0060", "`").replace("`", "``")
    result = tx.run(
                    """CALL gds.localClusteringCoefficient.stats('"""+escaped_graph_name+"""')
                    YIELD averageClusteringCoefficient, nodeCount""", graph_name = graph_name
                    )
    return list(result)

results = {}
for item in louv_comm_id_list:
    with driver.session(database="mytest") as session:
             result = session.execute_write(calculate_avg_сlustering_сoefficient, item)
             results[item] = result  


results

{8207: [<Record averageClusteringCoefficient=0.0 nodeCount=51>],
 8327: [<Record averageClusteringCoefficient=0.0 nodeCount=15>],
 8489: [<Record averageClusteringCoefficient=0.0 nodeCount=13>],
 8647: [<Record averageClusteringCoefficient=0.0 nodeCount=13>],
 8859: [<Record averageClusteringCoefficient=0.0 nodeCount=7>],
 2140: [<Record averageClusteringCoefficient=0.0 nodeCount=6>]}

Коэффициэнты кластеризации у всех сообществ равны нулю. Мы можем увидеть это и визуально - ни в одном сообществе нет "треугольников". Участники сообществ слабо связаны между собой - у них нет общих знакомых.

11. Напишем REST API для 

Данный API будет получать ФИО участника на входе, и на выходе отдавать JSON объект, содержащий ФИО, список событий в которых человек принимал участие и список знакомых.
Предполагается что в запросе будем получать url такого вида: /person/?name=Иванов+Иван+Иванович

In [37]:
app = Flask(__name__)  # инициируем объект приложения Flask

@app.route('/status', methods=['GET'])
def a_live():
    return "Alive!"



def search_query(tx, name):
    name = str(name)
    escaped_name = name.replace("\\u0060", "`").replace("`", "``")
    result = tx.run(
                    """MATCH(p:Person {name:'"""+escaped_name+"""'})
                        MATCH (f:Person)-[k:KNOWS]-(p) 
                        MATCH (p)-[r:PARTICIPATED]-(e:Event)
                        RETURN p AS person, f AS friends, e as events"""
                    )
    return result.data() #data() возвращает по факту список словарей


@app.route('/person/', methods=['GET'])
def search():
    error = None
    name = request.args.get('name')
    if name and name != '': #обработаем запрос, если он не пустой
        with driver.session(database="mytest") as session:
            result = session.execute_write(search_query, name)
            if result and result != '':
                #хоть data() обещает нам вернуть объект формата json, возвращает все равно список
                #уберем из него скобки
                return make_response(str(result).replace("[", "").replace("]", ""))
            else:
                return 'Персона по ФИО не найдена'
    else:
        error = 'Не введен запрос!'
        return 'Не введен запрос!'


app.run(port=5000)  #запустим веб приложениие на порту 5000. вписать можно любой СВОБОДНЫЙ порт

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Mar/2023 00:12:35] "GET /person/?name=Бугайчук+Роман+Эдуардович HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 00:12:48] "GET /person/?name=Бугайчук+Роман+Эдуардович HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 00:12:57] "GET /person/?name=Иванов+Иван+Иванович HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 00:13:02] "GET /person/ HTTP/1.1" 200 -
